# Miscellaneous Functions

The following are a few convenience functions.

# picker

The *picker* object allows to select non-contiguous values in cdms, for example let's say we want to retrieve only 850hPa and 200hPa from our `ta` variable.



In [1]:
import cdms2
ipsl_ta_file = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/ta/gr/v20180803/ta_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
ta = ipsl_ta_file("ta", time=("2000", "2010"), order='xtzy') # Just to make it hard
print("ta:", ta.shape)

ta: (144, 120, 19, 143)


Notice we have all possible levels:

In [2]:
print(ta.getLevel()[:])

[100000.  92500.  85000.  70000.  60000.  50000.  40000.  30000.  25000.
  20000.  15000.  10000.   7000.   5000.   3000.   2000.   1000.    500.
    100.]


With the `picker` we can retrieve exactly the two levels we need

In [3]:
import genutil
myPick = genutil.picker(level=[85000., 20000.])
ta = ipsl_ta_file('ta', myPick, time=("2000", "2010", 'con'))  # restrict levels
print(ta.getLevel()[:])

/opt/conda/lib/python3.7/site-packages/cdms2/fvariable.py:99: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[revlist]


[85000. 20000.]


If a level does not exist, you have the option to error exit (default),  *fill* with missing values or *skip* the missing coordinate value

In [4]:
# No 200. level in this data
myPick = genutil.picker(level=[85000., 200.], match=1) # All values must be present (default)
try:
    ta = ipsl_ta_file('ta', myPick, time=("2000", "2010", 'con'))  # restrict levels
except Exception as err:
    print("Could not retrieve selected coordinates\nerror was {}".format(err))
    
print("But this works")
myPick = genutil.picker(level=[85000., 200.], match=0)  # Fill with missing if necessary
ta = ipsl_ta_file('ta', myPick, time=("2000", "2010", 'con'))
print(ta.shape)

print("This works too, but will have one less level")
myPick = genutil.picker(level=[85000., 200.], match=-1)  # Fill with missing if necessary
ta = ipsl_ta_file('ta', myPick, time=("2000", "2010", 'con'))
print(ta.shape)

Could not retrieve selected coordinates
error was Error axis value :200.0 was requested but is not present in slab
(more missing might exists)
But this works
(120, 2, 143, 144)
This works too, but will have one less level
(120, 1, 143, 144)


# minmax

Returns the minimum and maximum of a series of arrays/lists/tuples (or a combination of these). You can combine list/tuples/… pretty much any combination is allowed.

Examples:

In [5]:
import genutil
s = range(7)
print(genutil.minmax(s))
print(genutil.minmax([4.,[6.,7.,s]],[5.,7.,8,(6.,1.)]))
print(genutil.minmax(ta))
print(genutil.minmax(ta*2., ta))
print(genutil.minmax(ta, s))

(0.0, 6.0)
(0.0, 8.0)
(223.70919799804688, 304.9276428222656)
(223.70919799804688, 1.0000000200408773e+20)
(0.0, 304.9276428222656)


# udunits

This is an historical module putting a python wrapper on top of UNIDATA's udunits, that was established before any other of the existing packages.

It allows to easily convert from one unit to another (if possible)

In [6]:
one_inch = genutil.udunits(1,"ft")
one_inch_in_meter = one_inch.to("m")
one_inch_in_meter

udunits(0.3048,"m")

You can query units convertion

In [7]:
print("Input units:", ta.units)
my_data = genutil.udunits(1, ta.units)
my_data.how("degC")

Input units: K


(1.0, -273.15)

In [8]:
print("Range:", genutil.minmax(ta))
conv = my_data.how("degF")
ta_F = ta*conv[0] + conv[1]
print("New range:", genutil.minmax(ta_F))

Range: (223.70919799804688, 304.9276428222656)
New range: (-56.99346923828125, 89.19973754882812)


# Filter

This includes very basic filtering, we have 3 filters: `smooth121`, `runningaverage`, `custom1D`

## Running Average

first let's produce a 2 years time serie

In [9]:
ta_time_serie = genutil.averager(ta[:24, 0], axis='xy')
print(ta_time_serie)

[278.3491413405935 278.68661400845673 279.25170587878335
 280.00351069508326 280.6275416850775 281.44136772781826 281.9891033021813
 281.9600884384358 281.129196790029 280.05186983911284 279.00521632975295
 278.31356543530666 278.279477889438 278.67549285230524 279.17431066504577
 279.8004320186885 280.7074459880988 281.55036455146086 282.0163765761603
 281.83819237756336 281.0523175336465 279.76357466970677 278.7958974045892
 278.2712075852329]


Now let's run a 12 months (yearly) running average.

`runnnigaverage` needs to know the number of consecutive steps to average together, 12 here

In [10]:
print(genutil.filters.runningaverage(ta_time_serie, 12))

[280.06741012 280.06160483 280.06067807 280.05422847 280.03730525
 280.04396394 280.05304701 280.05531978 280.04516178 280.03875517
 280.01473057 279.99728733 279.99375751]


## Custom 1D

One can define a custom filter by giving weights for each step, in our 12 month running average if would be:

In [11]:
yearly_running_average_filter = [1,1,1,1,1,1,1,1,1,1,1,1]
print(genutil.filters.custom1D(ta_time_serie, yearly_running_average_filter))

[280.06741012 280.06160483 280.06067807 280.05422847 280.03730525
 280.04396394 280.05304701 280.05531978 280.04516178 280.03875517
 280.01473057 279.99728733 279.99375751]


But we could choose to weight more the middle months

In [12]:
custom_yearly_running_average_filter = [1,2,2,3,3,4,4,3,3,2,2,1]
print(genutil.filters.custom1D(ta_time_serie, custom_yearly_running_average_filter))

[280.51788053 280.52202612 280.40650676 280.19179229 279.94212844
 279.72266406 279.59422122 279.58515586 279.69723406 279.89960678
 280.13390145 280.33901219 280.4610455 ]


## Smooth121

Essentially a custom filter with weight: [1,2,1]

In [13]:
print(genutil.filters.custom1D(ta_time_serie, [1,2,1]))
print(genutil.filters.smooth121(ta_time_serie))

[278.74351881 279.29838412 279.97156724 280.67499045 281.37484511
 281.84491569 281.75961924 281.06758796 280.0595382  279.09396698
 278.47795627 278.38700352 278.70119356 279.20613655 279.87065517
 280.69142214 281.45613792 281.85532752 281.68626972 280.92660053
 279.84384107 278.90664427]
[278.74351881 279.29838412 279.97156724 280.67499045 281.37484511
 281.84491569 281.75961924 281.06758796 280.0595382  279.09396698
 278.47795627 278.38700352 278.70119356 279.20613655 279.87065517
 280.69142214 281.45613792 281.85532752 281.68626972 280.92660053
 279.84384107 278.90664427]
